In [ ]:
import pandas
import numpy
import matplotlib.pyplot as plt
import matplotlib
import sys
import os
import pyarrow.parquet as pq
import plotly
import folium
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
import plotly.offline as py
import plotly.graph_objs as go
import geopandas as gpd

In [ ]:
# donnees_incidents.csv : Client_AI	Client_AQN	PS_HTA_AI	PS_HTA_AQN	AnnÃ©e	Mois	Mois (nom)	AnnÃ©e-Mois	Semaine	jour	ADEL	ADEL/Exceptionnel	CPI	DurÃ©e >= 3mn	INSEE	BO	Territoire	Clients par dÃ©part HTA	Poste Source	Code centre	Date d'interruption	Heure d'Interruption (hhmm)	Code du dÃ©part HTA	Nature du Jour	Cause de l'interruption	SiÃ¨ge de l'Interruption	DurÃ©e Totale Interruption (min)	DurÃ©e de Localisation (min)	DurÃ©e de DÃ©pannage (min)	DurÃ©e de retour Ã  la normale (min)	Nb renvois en phase localisation	Fonctionnement appareil coupure	Type interruption exceptionnelle	Lieu de dÃ©faut code GDO objet 1	Lieu de dÃ©faut code GDO objet 2	id	x	y	CENTRE	STATUT	DATE_DE_CO	TENSION_D_	NOMBRE_DE_	UNITÃ‰_D_E	DÃ‰SIGNATI	LONGUEUR_	ID_SIG	CRÃ‰Ã‰_LE	MIS_Ã€_JOU	TECHNOLOGI	INTENSITÃ‰	DÃ‰SIGNAT1	COMMUNE	INTENSIT1	new_x	new_y	new_xbis	new_ybis
# souterrain_2023.csv : WKT	CENTRE	STATUT	DATE_DE_CO	TENSION_D_	NOMBRE_DE_	UNITÃ‰_D_E	DÃ‰SIGNATI	LONGUEUR_	ID_SIG	CRÃ‰Ã‰_LE	MIS_Ã€_JOU	DOSSIER_MO	TECHNOLOGI	INTENSITÃ‰	DÃ‰SIGNAT1	COMMUNE	CODE_GDO	INTENSIT1	STATUT_DEP	WKT_next	ID_SIG_next	x0	y0	x1	y1
# souterrain_troncons.csv : WKT	CENTRE	STATUT	DATE_DE_CO	TENSION_D_	NOMBRE_DE_	UNITÃ‰_D_E	DÃ‰SIGNATI	LONGUEUR_	ID_SIG	CRÃ‰Ã‰_LE	MIS_Ã€_JOU	DOSSIER_MO	TECHNOLOGI	INTENSITÃ‰	DÃ‰SIGNAT1	COMMUNE	CODE_GDO	INTENSIT1	STATUT_DEP
# icu.csv : WKT	fid	TEMP_1	UU_1
# temperatures.parquet
path = "data/"
# ouvrir les fichiers
incidents = pandas.read_csv(path+'donnees_incidents.csv', sep=';')
souterrain_2023 = pandas.read_csv(path+'souterrain_2023.csv', sep=';')
souterrain_troncons = pandas.read_csv(path+'souterrain_tronçons.csv', sep=';')
icu = pandas.read_csv(path+'icu.csv', sep=';')
temperatures = pq.read_table(path+'temperatures.parquet').to_pandas()

# plotly show number of incidents per month [january to december]
incidents_per_month = incidents.groupby('Mois (nom)').size()
# order months by chronological order
incidents_per_month = incidents_per_month.reindex(['janv', 'févr', 'mars', 'avr', 'mai', 'juin', 'juil', 'août', 'sept', 'oct', 'nov', 'déc'])
incidents_per_month.plot.bar()
plt.title('Number of incidents per month')
plt.xlabel('Month')
plt.ylabel('Number of incidents')
plt.show()

In [ ]:
# show incidents on a map
# create a map
m = folium.Map(location=[45.75, 4.85], zoom_start=12)
# add incidents
for index, row in incidents.iterrows():
    try :
        folium.Marker([row['new_y'], row['new_x']], popup=row['Cause de l\'interruption']).add_to(m)
    except :
        print("error: ", row['new_x'], row['new_y'])
        pass
# add underground cables
for index, row in souterrain_2023.iterrows():
    try : 
        folium.PolyLine([[row['y0'], row['x0']], [row['y1'], row['x1']]], color="red", weight=2.5, opacity=1).add_to(m)
    except :
        print("error: ", row['x0'], row['y0'], row['x1'], row['y1'])
        pass


In [ ]:
# plot the map
m.save('map.html')

In [ ]:
print(temperatures)
# date_valid     valeur  longitude  latitude
# 0       2014-01-01 00:00:00   6.712457        0.1      44.0
# 1       2014-01-01 01:00:00   6.176080        0.1      44.0

In [ ]:
# xgboost model to predict if there will be an incident or not on a given day (taking into account temperature)

# create a dataframe with the number of incidents per day (Année-Mois |  Semaine |  jour) and fill with 0 when there is no incident on a given day
incidents_per_day = incidents.groupby(['Année-Mois', 'Semaine', 'jour']).size()


incidents_per_day = incidents_per_day.reset_index()
incidents_per_day = incidents_per_day.rename(columns={0: 'Value'})
incidents_per_day['Année-Mois-jour'] = incidents_per_day['Année-Mois'] + '-' + incidents_per_day['jour'].astype(str)
incidents_per_day = incidents_per_day.drop(columns=['Année-Mois', 'Semaine', 'jour'])
incidents_per_day = incidents_per_day.set_index('Année-Mois-jour')

# add missing days with 0 value
start_date = incidents_per_day.index[0]
end_date = incidents_per_day.index[-1]
idx = pandas.date_range(start_date, end_date)
incidents_per_day.index = pandas.DatetimeIndex(incidents_per_day.index)
incidents_per_day = incidents_per_day.reindex(idx, fill_value=0)
incidents_per_day = incidents_per_day.reset_index()
incidents_per_day = incidents_per_day.rename(columns={'index': 'Année-Mois-jour'})
incidents_per_day['Année-Mois-jour'] = incidents_per_day['Année-Mois-jour'].astype(str)
incidents_per_day['Année-Mois-jour'] = incidents_per_day['Année-Mois-jour'].str.replace(' 00:00:00', '')
incidents_per_day = incidents_per_day.set_index('Année-Mois-jour')
incidents_per_day = incidents_per_day.sort_index()


plt.plot(incidents_per_day)
plt.title('Number of incidents per day')
plt.xlabel('Date')
plt.ylabel('Number of incidents')
plt.show()

#show histogram of accident
incidents_per_day.hist(bins= range(10))
plt.title('Histogram of number of incidents per day')
plt.xlabel('Number of incidents')
plt.ylabel('Number of days')
plt.show()



# create a dataframe with the temperature per day
temperatures_per_day = temperatures.groupby('date_valid').mean()
temperatures_per_day = temperatures_per_day.drop(columns=['longitude', 'latitude'])
temperatures_per_day.index = pandas.to_datetime(temperatures_per_day.index)
temperatures_per_day = temperatures_per_day.resample('D').mean()
temperatures_per_day = temperatures_per_day.dropna()

# plot mean temperature per day 
plt.plot(temperatures_per_day)
plt.title('Mean temperature per day')
plt.xlabel('Date')
plt.ylabel('Mean temperature')
plt.show()



